In [60]:
import requests
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
import time
import json
import requests
from requests.exceptions import SSLError
import psycopg2
import random
import csv
from email.mime.text import MIMEText
from email.utils import COMMASPACE
import smtplib

headers = {
    'authority': 'moto.av.by',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ru,en;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "YaBrowser";v="24.1", "Yowser";v="2.5"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 YaBrowser/24.1.0.0 Safari/537.36',
}

# Чтение json конфига
with open('config.json') as file:
    config = json.load(file)

mail_login = config['sender login']
mail_password = config['sender password']
pgre_login = config['postgre login']
pgre_password = config['postgre password']
pgre_host = config['postgre host']
pgre_port = config['postgre port']
pgre_db = config['postgre database']
recipients = config['mail recipients']

#Подключение к postgres
conn = psycopg2.connect(
    host = pgre_host,
    port = pgre_port,
    database = pgre_db,
    user = pgre_login,
    password = pgre_password
)


current_time_start = datetime.now()
ctsf = current_time_start.strftime("%Y-%m-%d %H:%M:%S")
print(f"Привет! Текущая дата - {ctsf}")

Привет! Текущая дата - 2024-10-25 15:21:05


In [61]:
cursor = conn.cursor()

In [3]:
id_value = 112262325 # хач бмв 112262309 #эндурик 110511290

In [4]:
# Ищем дубликат по id_value, список не будет содержать строку искомого айди
select_query = """SELECT id, date, price, model_vlk, brand, model, model_misc, year, type, cylinders, capacity, mileage, exclude_flag, url
FROM public.av_full
WHERE (brand, model, model_misc, year, type, cylinders, capacity, mileage, seller, locations) = 
(
    SELECT brand, model, model_misc, year, type, cylinders, capacity, mileage, seller, locations
    FROM public.av_full
    WHERE id = %d
)
AND id != %d
ORDER BY date ASC;""" % (id_value, id_value)
cursor.execute(select_query)
rows = cursor.fetchall()


In [5]:
if rows:
    dupl_id_list = []
    dupl_date_list = []
    for n in rows:    
        dupl_dates = n[1]
        dupl_date_list.append(dupl_dates)
        dupl_id = n[0]
        dupl_id_list.append(dupl_id)
        print(f"id - {dupl_id}, дата - {dupl_dates}")

    dupl_date = min(dupl_date_list)
    
    print(f"дата - {dupl_date}")
else:
    print('нетути')

# в итоге у нас получатся что dupl_id_list содержит все дубликаты для id_value. dupl_date_list содержит список дат, а dupl_date самую раннюю

id - 111343735, дата - 2024-10-08 12:39:48
дата - 2024-10-08 12:39:48


In [6]:
dupl_id_list = ', '.join(str(int(d)) for d in dupl_id_list)

In [7]:
# писать кандишн - Дубликат в список айди и писать новую дату в id_value 


# выставить флажок дубликата
query1 = """UPDATE public.av_full
SET duplicate_flag = True
where id in (%s);""" % (dupl_id_list)
cursor.execute(query1)

# выставить новую дату
query2 = """UPDATE public.av_full
SET date = '%s'
where id = %i;""" % (dupl_date, id_value)
cursor.execute(query2)


In [22]:
conn.commit()
#conn.close()

In [ ]:
#он по идее должен вернуть массивчик если есть дубли или нул если дублей нет
#если дубли есть то взять самую старую дату и заменить у искомого айди, а все в списке пометить флажочком
#строка 180 в актчек

In [107]:

def check_for_duplicates (id_value):
    # Ищем дубликат по id_value, список не будет содержать строку искомого айди
    select_query = """SELECT id, date, price, model_vlk, brand, model, model_misc, year, type, cylinders, capacity, mileage, exclude_flag, url
    FROM public.av_full
    WHERE (brand, model, model_misc, year, type, cylinders, capacity, mileage, seller, locations) = 
    (
        SELECT brand, model, model_misc, year, type, cylinders, capacity, mileage, seller, locations
        FROM public.av_full
        WHERE id = %d
    )
    AND id != %d
    ORDER BY date ASC;""" % (id_value, id_value)
    cursor.execute(select_query)
    rows = cursor.fetchall()
    dupl_count = 0 # счетчик количества объяв-дубликатов

    if rows:
        dupl_id_list = []
        dupl_date_list = []
    
        for n in rows:    
            dupl_dates = n[1]
            dupl_date_list.append(dupl_dates)
            dupl_id = n[0]
            dupl_id_list.append(dupl_id)
            #print(f"id - {dupl_id}, дата - {dupl_dates}")
            dupl_count += 1

        dupl_date = min(dupl_date_list)
        
        print(f"Найдено {dupl_count} дубликатов для id:{id_value} с самой ранней датой - {dupl_date}")

        # отформатировать список для вставки в квери
        dupl_id_list = ', '.join(str(int(d)) for d in dupl_id_list)

        # квери выставить флажок дубликата
        query1 = """UPDATE public.av_full
        SET duplicate_flag = True, duplicate_id = %d
        WHERE id in (%s);""" % (id_value, dupl_id_list)
        cursor.execute(query1)

        # квери выставить новую дату
        query2 = """UPDATE public.av_full
        SET date = '%s'
        WHERE id = %d;""" % (dupl_date, id_value)
        cursor.execute(query2)
        
        # записать изменения
        ##############################################################################################conn.commit()
        return 1
    else:
        print(f'Дубликатов для id:{id_value} не обнаружено')
        return 0
    


In [108]:
duplicates_global_count = 0

In [117]:
id_value = 112262325 #112262325 # хач бмв 112262309 #эндурик 110511290
seller = 'Василий' #"Продажа мотоциклов и прицеп дач Вязынка"
capacity = 1000
cylcount = 4
brand = 'Kawasaki'
duplicate_flag = False #!!!! доставать его из базы


In [118]:
if seller != 'Продажа мотоциклов и прицеп дач Вязынка' and int(capacity) >= 299 and cylcount > 1 and brand not in ('Днепр', 'Jawa', 'ИЖ', 'Эксклюзив', 'Racer', 'Урал', 'Cezet') and duplicate_flag is False:
    duplicates_global_count += check_for_duplicates (id_value)    
else:
    print(f'Проверка дубликатов для id:{id_value} не проводится')

Найдено 1 дубликатов для id:112262325 с самой ранней датой - 2024-10-08 12:39:48


In [119]:
duplicates_global_count

2